In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
tf.reset_default_graph()

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pickle

training_data = pickle.load(open("data_set1.pickle", "rb"))

training_data = training_data/255.0

In [ ]:
ruido_ph = tf.placeholder(tf.float32,[None, 100])

In [ ]:
def gerador(ruido, reuse = None):
    with tf.variable_scope('gerador', reuse = reuse):
        #100 -> 128 -> 128 -> 784
        camada_oculta1 = tf.nn.relu(tf.layers.dense(inputs = ruido, units=1280))
        camada_oculta2 = tf.nn.relu(tf.layers.dense(inputs=camada_oculta1, units=1280))
        camada_saida = tf.layers.dense(inputs = camada_oculta2, units = 9216, activation = tf.nn.tanh)
        return camada_saida

In [ ]:
imagens_reais_ph = tf.placeholder(tf.float32, [None, 9216])

In [ ]:
def discriminador(X, reuse = None):
    with tf.variable_scope('discriminador', reuse = reuse):
        # 784 -> 128 -> 128 -> 1
        camada_oculta1 = tf.nn.relu(tf.layers.dense(inputs = X, units = 1280))
        camada_oculta2 = tf.nn.relu(tf.layers.dense(inputs = camada_oculta1, units = 1280))
        logits = tf.layers.dense(camada_oculta2, units=1)
        return logits

In [ ]:
logits_imagens_reais = discriminador(imagens_reais_ph)
logits_imagens_ruido = discriminador(gerador(ruido_ph), reuse = True)

erro_discriminador_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits_imagens_reais,
                                                                                labels = tf.ones_like(logits_imagens_reais)*(0.9)))

erro_discriminador_ruido = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits_imagens_ruido,
                                                                                 labels = tf.zeros_like(logits_imagens_ruido)))

erro_discriminador = erro_discriminador_real + erro_discriminador_ruido

erro_gerador = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits_imagens_ruido,
                                                                     labels = tf.ones_like(logits_imagens_ruido)))

In [ ]:
variaveis = tf.trainable_variables()

In [ ]:
variaveis_discriminador = [v for v in variaveis if 'discriminador' in v.name]

In [ ]:
variaveis_gerador = [v for v in variaveis if 'gerador' in v.name]

In [ ]:
treinamento_discriminador = tf.train.AdamOptimizer(learning_rate=0.001).minimize(erro_discriminador,
                                                                                var_list = variaveis_discriminador)
treinamento_gerador = tf.train.AdamOptimizer(learning_rate=0.001).minimize(erro_gerador, var_list=variaveis_gerador)

In [ ]:
batch_size =  100
amostras_teste = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #ruido_teste = np.random.uniform(-1, 1, size =(1,100))
    #amostra = sess.run(gerador(ruido_ph, reuse=True), feed_dict = {ruido_ph: ruido_teste})
    
#     batch = mnist.train.next_batch(100)
#     imagens_batch = batch[0].reshape((100, 784))
#     imagens_batch = imagens_batch*2 - 1
#     r = sess.run(discriminador(imagens_reais_ph, True), feed_dict = {imagens_reais_ph: imagens_batch})
    
    for epoca in range(5000):
        numero_batches = len(training_data) // batch_size
        for i in range(numero_batches):
            batch = training_data[batch_size*i:batch_size*i + batch_size]
            imagens_batch = batch.reshape((batch_size,9216))
            imagens_batch = imagens_batch * 2 - 1
    
            batch_ruido = np.random.uniform(-1, 1, size=(batch_size,100))
        
            _, custod = sess.run([treinamento_discriminador, erro_discriminador],
                                  feed_dict  = {imagens_reais_ph: imagens_batch, ruido_ph: batch_ruido})
            
            _, custog = sess.run([treinamento_gerador, erro_gerador], feed_dict = {ruido_ph: batch_ruido})
            
        print('época: ' + str(epoca + 1) + ' erro D: '+str(custod)+' erro G: '+str(custog))
        
        if((epoca+1)%500 == 0):
            # A cada 500 épocas, salva uma imagem
            ruido_teste = np.random.uniform(-1, 1, size=(1,100))
            imagem_gerada = sess.run(gerador(ruido_ph, reuse = True), feed_dict={ruido_ph: ruido_teste})

            amostras_teste.append(imagem_gerada)

In [ ]:
plt.imshow(amostras_teste[9].reshape(96,96), cmap = 'Greys')